In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

# load the Tiny Shakespeare dataset
dataset, info = tfds.load('tiny_shakespeare', with_info=True, as_supervised=False)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\Dell\tensorflow_datasets\tiny_shakespeare\incomplete.SCIKVK_1.0.0\tiny_shakespeare-train.tf…

Generating validation examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\Dell\tensorflow_datasets\tiny_shakespeare\incomplete.SCIKVK_1.0.0\tiny_shakespeare-validati…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\Dell\tensorflow_datasets\tiny_shakespeare\incomplete.SCIKVK_1.0.0\tiny_shakespeare-test.tfr…

Dataset tiny_shakespeare downloaded and prepared to C:\Users\Dell\tensorflow_datasets\tiny_shakespeare\1.0.0. Subsequent calls will reuse this data.


In [3]:
# get the text from the dataset
text = next(iter(dataset['train']))['text'].numpy().decode('utf-8')

# create a mapping from unique characters to indices
vocab = sorted(set(text))
char2idx = {char: idx for idx, char in enumerate(vocab)}
idx2char = np.array(vocab)

# numerically represent the characters
text_as_int = np.array([char2idx[c] for c in text])

# create training examples and targets
seq_length = 100
examples_per_epoch = len(text) // (seq_length + 1)

# create training sequences
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

sequences = char_dataset.batch(seq_length + 1, drop_remainder=True)

In [4]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [5]:
# batch size and buffer size
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE)
)

In [6]:
# length of the vocabulary
vocab_size = len(vocab)

# the embedding dimension
embedding_dim = 256

# number of RNN units
rnn_units = 1024

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
        tf.keras.layers.LSTM(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

model = build_model(vocab_size, embedding_dim, rnn_units, BATCH_SIZE)

In [7]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer='adam', loss=loss)

In [8]:
import os

# directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'

# name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True
)

# train the model
EPOCHS = 10
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
155/155 [==============================] - 2316s 15s/step - loss: 2.6402
Epoch 2/10
155/155 [==============================] - 819s 5s/step - loss: 1.9335
Epoch 3/10
155/155 [==============================] - 813s 5s/step - loss: 1.6720
Epoch 4/10
155/155 [==============================] - 883s 6s/step - loss: 1.5318
Epoch 5/10
155/155 [==============================] - 802s 5s/step - loss: 1.4445
Epoch 6/10
155/155 [==============================] - 835s 5s/step - loss: 1.3849
Epoch 7/10
155/155 [==============================] - 851s 5s/step - loss: 1.3372
Epoch 8/10
155/155 [==============================] - 1620s 10s/step - loss: 1.2971
Epoch 9/10
155/155 [==============================] - 864s 6s/step - loss: 1.2613
Epoch 10/10
155/155 [==============================] - 763s 5s/step - loss: 1.2255


In [9]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [10]:
def generate_text(model, start_string):
    num_generate = 1000

    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []

    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)

        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

print(generate_text(model, start_string=u"QUEEN: So, lets end this"))

QUEEN: So, lets end this desire it;
May I swear me brought and dargeny.
Where is the promise that falls out
Of all commission, fled, he will preverent,
After every gracious earth to have been proud, Hereford, as I see
The hopey of my name is drawd.

ROMEO:
Struck my head of me!
Let me mistress us and that weigh of ither in right,
To stir is me inquiret, come and thundark.

DUKE VINCENTIO:
Blay name is came framed, if I may hear in
as fallef wretch;' and I not die.

DEKBe in his manch of death is all:
Leave us we will bet, in rages. Lords, lords, to greet dead a
woman's blessenge; then,
Ere the bit of their wings
And beaute whose words have done his dream.

KING EDWARD IV:
But how Our Warwick, in the grace To bear a law,
The man will be: but I can reven them wom,
And now false lastage to my hellow mocked wounds.

SICINIUS:
Why, so I have done so;
Lace not with notisful whit enough.

CAMILLO:
Why, more time; at your head.

DURE VINCENTIO:
Sir, I know the parcel, offen of right and shame?